In [70]:
import numpy as np
import pandas as pd
import math

In [2]:
df = pd.read_csv('YCombinatorStartups.csv', index_col=0)

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
df.head()

,id,name,slug,former_names,small_logo_thumb_url,website,all_locations,long_description,one_liner,team_size,highlight_black,highlight_latinx,highlight_women,industry,subindustry,launched_at,tags,tags_highlighted,top_company,top_company_by_revenue,isHiring,nonprofit,batch,status,industries,regions,stage,app_video_public,demo_day_video_public,app_answers,question_answers,objectID
0,325,Dropbox,dropbox,[],https://bookface-images.s3.amazonaws.com/small...,http://dropbox.com,"San Francisco, CA, USA",Dropbox is building the world’s first smart wo...,Backup and share files in the cloud.,4000.0,False,False,False,B2B,B2B -> Productivity,1326791328,[],[],True,True,False,False,S07,Public,"['B2B', 'Productivity']","['United States of America', 'America / Canada']",Growth,False,False,NaN,False,325
1,379,Reddit,reddit,[],https://bookface-images.s3.amazonaws.com/small...,http://reddit.com,"San Francisco, CA, USA",Founded by Steve Huffman and Alexis Ohanian in...,The frontpage of the internet.,201.0,False,False,False,Consumer,Consumer -> Content,1326791708,"['Community', 'Social Media', 'Social', 'Socia...",[],True,True,False,False,S05,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Early,False,False,NaN,False,379
2,383,Twitch,twitch,"['Justin.tv', 'Twitch']",https://bookface-images.s3.amazonaws.com/small...,http://twitch.com,"San Francisco, CA, USA",Twitch is the world’s leading video platform a...,A global community creating the future of live...,2000.0,True,False,False,Consumer,Consumer -> Content,1326791723,"['Community', 'Gaming', 'Social Media', 'Video...",[],True,False,False,False,W07,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada...",Growth,False,False,NaN,False,383
3,356,Scribd,scribd,[],https://bookface-images.s3.amazonaws.com/small...,http://scribd.com,"San Francisco, CA, USA",Read and listen without limits. Unlimited* aud...,World's largest online library.,300.0,False,False,False,Consumer,Consumer -> Content,1326791580,[],[],True,True,True,False,S06,Active,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Growth,False,False,NaN,False,356
4,344,Weebly,weebly,[],https://bookface-images.s3.amazonaws.com/small...,http://weebly.com,"San Francisco, CA, USA",Company Information\r\nWeebly is a consumer se...,Build a free website that grows with your busi...,201.0,False,False,False,B2B,B2B -> Marketing,1326791493,[],[],True,False,False,False,W07,Acquired,"['B2B', 'Marketing']","['United States of America', 'America / Canada']",Growth,False,False,NaN,False,344


In [5]:
df.drop(columns=['id', 'slug', 'objectID', 'app_answers', 'website'], inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4410 entries, 0 to 4409
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    4410 non-null   object 
 1   former_names            4410 non-null   object 
 2   small_logo_thumb_url    4410 non-null   object 
 3   all_locations           4253 non-null   object 
 4   long_description        4072 non-null   object 
 5   one_liner               4216 non-null   object 
 6   team_size               4344 non-null   float64
 7   highlight_black         4410 non-null   bool   
 8   highlight_latinx        4410 non-null   bool   
 9   highlight_women         4410 non-null   bool   
 10  industry                4410 non-null   object 
 11  subindustry             4410 non-null   object 
 12  launched_at             4410 non-null   int64  
 13  tags                    4410 non-null   object 
 14  tags_highlighted        4410 non-null   objec

In [7]:
df['num_former_names'] = df['former_names'].apply(lambda row: 0 if len(row) == 2 else len(row.replace('[', '').replace(']', '').replace("'", "").replace(' ', '').split(',')))

In [8]:
df.head()

,name,former_names,small_logo_thumb_url,all_locations,long_description,one_liner,team_size,highlight_black,highlight_latinx,highlight_women,industry,subindustry,launched_at,tags,tags_highlighted,top_company,top_company_by_revenue,isHiring,nonprofit,batch,status,industries,regions,stage,app_video_public,demo_day_video_public,question_answers,num_former_names
0,Dropbox,[],https://bookface-images.s3.amazonaws.com/small...,"San Francisco, CA, USA",Dropbox is building the world’s first smart wo...,Backup and share files in the cloud.,4000.0,False,False,False,B2B,B2B -> Productivity,1326791328,[],[],True,True,False,False,S07,Public,"['B2B', 'Productivity']","['United States of America', 'America / Canada']",Growth,False,False,False,0
1,Reddit,[],https://bookface-images.s3.amazonaws.com/small...,"San Francisco, CA, USA",Founded by Steve Huffman and Alexis Ohanian in...,The frontpage of the internet.,201.0,False,False,False,Consumer,Consumer -> Content,1326791708,"['Community', 'Social Media', 'Social', 'Socia...",[],True,True,False,False,S05,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Early,False,False,False,0
2,Twitch,"['Justin.tv', 'Twitch']",https://bookface-images.s3.amazonaws.com/small...,"San Francisco, CA, USA",Twitch is the world’s leading video platform a...,A global community creating the future of live...,2000.0,True,False,False,Consumer,Consumer -> Content,1326791723,"['Community', 'Gaming', 'Social Media', 'Video...",[],True,False,False,False,W07,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada...",Growth,False,False,False,2
3,Scribd,[],https://bookface-images.s3.amazonaws.com/small...,"San Francisco, CA, USA",Read and listen without limits. Unlimited* aud...,World's largest online library.,300.0,False,False,False,Consumer,Consumer -> Content,1326791580,[],[],True,True,True,False,S06,Active,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Growth,False,False,False,0
4,Weebly,[],https://bookface-images.s3.amazonaws.com/small...,"San Francisco, CA, USA",Company Information\r\nWeebly is a consumer se...,Build a free website that grows with your busi...,201.0,False,False,False,B2B,B2B -> Marketing,1326791493,[],[],True,False,False,False,W07,Acquired,"['B2B', 'Marketing']","['United States of America', 'America / Canada']",Growth,False,False,False,0


In [9]:
df.drop(columns=['former_names', 'small_logo_thumb_url', 'industry', 'subindustry', 'launched_at'], inplace=True)

In [18]:
df['batch'].value_counts()

batch
W22     402
S21     393
W21     337
W23     272
S22     235
W20     227
S23     218
S20     208
W19     197
S19     174
W18     147
S18     132
S17     125
W16     123
W17     116
W15     111
S15     105
S16     102
S12      82
S14      79
IK12     77
W14      74
W12      66
S11      60
S13      52
W13      46
W11      45
S10      36
W10      26
S09      26
S08      22
W08      21
S07      19
W09      16
W07      13
S06      11
S05       8
W06       7
Name: count, dtype: int64

In [20]:
df['batch'].apply(lambda row: int(row.replace('W', '').replace('S', '').replace('IK', '')))

0        7
1        5
2        7
3        6
4        7
        ..
4405    22
4406    23
4407    23
4408    22
4409    22
Name: batch, Length: 4410, dtype: int64

In [21]:
df['years_since_founding'] = df['batch'].apply(lambda row: 23 - int(row.replace('W', '').replace('S', '').replace('IK', '')))

In [25]:
df.drop(columns='batch', inplace=True)

In [30]:
df.head()

,name,all_locations,long_description,one_liner,team_size,highlight_black,highlight_latinx,highlight_women,tags,tags_highlighted,top_company,top_company_by_revenue,isHiring,nonprofit,status,industries,regions,stage,app_video_public,demo_day_video_public,question_answers,num_former_names,years_since_founding
0,Dropbox,"San Francisco, CA, USA",Dropbox is building the world’s first smart wo...,Backup and share files in the cloud.,4000.0,False,False,False,[],[],True,True,False,False,Public,"['B2B', 'Productivity']","['United States of America', 'America / Canada']",Growth,False,False,False,0,16
1,Reddit,"San Francisco, CA, USA",Founded by Steve Huffman and Alexis Ohanian in...,The frontpage of the internet.,201.0,False,False,False,"['Community', 'Social Media', 'Social', 'Socia...",[],True,True,False,False,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Early,False,False,False,0,18
2,Twitch,"San Francisco, CA, USA",Twitch is the world’s leading video platform a...,A global community creating the future of live...,2000.0,True,False,False,"['Community', 'Gaming', 'Social Media', 'Video...",[],True,False,False,False,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada...",Growth,False,False,False,2,16
3,Scribd,"San Francisco, CA, USA",Read and listen without limits. Unlimited* aud...,World's largest online library.,300.0,False,False,False,[],[],True,True,True,False,Active,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Growth,False,False,False,0,17
4,Weebly,"San Francisco, CA, USA",Company Information\r\nWeebly is a consumer se...,Build a free website that grows with your busi...,201.0,False,False,False,[],[],True,False,False,False,Acquired,"['B2B', 'Marketing']","['United States of America', 'America / Canada']",Growth,False,False,False,0,16


In [119]:
df['all_locations'].value_counts()

all_locations
San Francisco, CA, USA                                                                 912
San Francisco, CA, USA; Remote                                                         286
New York, NY, USA                                                                      228
Bengaluru, KA, India                                                                    86
New York, NY, USA; Remote                                                               82
                                                                                      ... 
United States; Portland, OR, USA; Chattanooga, TN, USA; Remote                           1
Toronto, ON, Canada; London, England, United Kingdom; Vancouver, BC, Canada; Remote      1
Santa Clara, CA, USA; Sunnyvale, CA, USA                                                 1
Denver, CO, USA; Austin, TX, USA                                                         1
College Park, MD, USA; Rockville, MD, USA; Remote                           

In [98]:
df.drop(columns='tags_highlighted', inplace=True)

In [105]:
df['all_locations'][33]

nan

In [120]:
df['all_locations'][4406]

'San Francisco, CA, USA; Sunnyvale, CA, USA; Remote'

In [121]:
[x.lstrip() for x in df['all_locations'][4406].split(';')]

['San Francisco, CA, USA', 'Sunnyvale, CA, USA', 'Remote']

In [125]:
df['locations'] = df['all_locations'].apply(lambda row: row.split(';') if type(row)==str else [])

In [129]:
for idx, row in df.iterrows():
    df.at[idx, 'locations'] = [x.lstrip() for x in row['locations']]
df.head()

,name,all_locations,long_description,one_liner,team_size,highlight_black,highlight_latinx,highlight_women,tags,top_company,top_company_by_revenue,isHiring,nonprofit,status,industries,regions,stage,app_video_public,demo_day_video_public,question_answers,num_former_names,years_since_founding,locations
0,Dropbox,"San Francisco, CA, USA",Dropbox is building the world’s first smart wo...,Backup and share files in the cloud.,4000.0,False,False,False,[],True,True,False,False,Public,"['B2B', 'Productivity']","['United States of America', 'America / Canada']",Growth,False,False,False,0,16,"[San Francisco, CA, USA]"
1,Reddit,"San Francisco, CA, USA",Founded by Steve Huffman and Alexis Ohanian in...,The frontpage of the internet.,201.0,False,False,False,"['Community', 'Social Media', 'Social', 'Socia...",True,True,False,False,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Early,False,False,False,0,18,"[San Francisco, CA, USA]"
2,Twitch,"San Francisco, CA, USA",Twitch is the world’s leading video platform a...,A global community creating the future of live...,2000.0,True,False,False,"['Community', 'Gaming', 'Social Media', 'Video...",True,False,False,False,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada...",Growth,False,False,False,2,16,"[San Francisco, CA, USA]"
3,Scribd,"San Francisco, CA, USA",Read and listen without limits. Unlimited* aud...,World's largest online library.,300.0,False,False,False,[],True,True,True,False,Active,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Growth,False,False,False,0,17,"[San Francisco, CA, USA]"
4,Weebly,"San Francisco, CA, USA",Company Information\r\nWeebly is a consumer se...,Build a free website that grows with your busi...,201.0,False,False,False,[],True,False,False,False,Acquired,"['B2B', 'Marketing']","['United States of America', 'America / Canada']",Growth,False,False,False,0,16,"[San Francisco, CA, USA]"


In [130]:
df['locations'].value_counts()

locations
[San Francisco, CA, USA]                                                                 912
[San Francisco, CA, USA, Remote]                                                         286
[New York, NY, USA]                                                                      228
[]                                                                                       157
[Bengaluru, KA, India]                                                                    86
                                                                                        ... 
[United States, Portland, OR, USA, Chattanooga, TN, USA, Remote]                           1
[Toronto, ON, Canada, London, England, United Kingdom, Vancouver, BC, Canada, Remote]      1
[Santa Clara, CA, USA, Sunnyvale, CA, USA]                                                 1
[Denver, CO, USA, Austin, TX, USA]                                                         1
[College Park, MD, USA, Rockville, MD, USA, Remote]         

In [131]:
df.explode('locations')

,name,all_locations,long_description,one_liner,team_size,highlight_black,highlight_latinx,highlight_women,tags,top_company,top_company_by_revenue,isHiring,nonprofit,status,industries,regions,stage,app_video_public,demo_day_video_public,question_answers,num_former_names,years_since_founding,locations
0,Dropbox,"San Francisco, CA, USA",Dropbox is building the world’s first smart wo...,Backup and share files in the cloud.,4000.0,False,False,False,[],True,True,False,False,Public,"['B2B', 'Productivity']","['United States of America', 'America / Canada']",Growth,False,False,False,0,16,"San Francisco, CA, USA"
1,Reddit,"San Francisco, CA, USA",Founded by Steve Huffman and Alexis Ohanian in...,The frontpage of the internet.,201.0,False,False,False,"['Community', 'Social Media', 'Social', 'Socia...",True,True,False,False,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Early,False,False,False,0,18,"San Francisco, CA, USA"
2,Twitch,"San Francisco, CA, USA",Twitch is the world’s leading video platform a...,A global community creating the future of live...,2000.0,True,False,False,"['Community', 'Gaming', 'Social Media', 'Video...",True,False,False,False,Acquired,"['Consumer', 'Content']","['United States of America', 'America / Canada...",Growth,False,False,False,2,16,"San Francisco, CA, USA"
3,Scribd,"San Francisco, CA, USA",Read and listen without limits. Unlimited* aud...,World's largest online library.,300.0,False,False,False,[],True,True,True,False,Active,"['Consumer', 'Content']","['United States of America', 'America / Canada']",Growth,False,False,False,0,17,"San Francisco, CA, USA"
4,Weebly,"San Francisco, CA, USA",Company Information\r\nWeebly is a consumer se...,Build a free website that grows with your busi...,201.0,False,False,False,[],True,False,False,False,Acquired,"['B2B', 'Marketing']","['United States of America', 'America / Canada']",Growth,False,False,False,0,16,"San Francisco, CA, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407,Mogara,"San Francisco, CA, USA; Remote",Mogara automates software R&D capitalization.\...,Automatic software R&D capitalization,2.0,False,False,True,"['Finance', 'B2B']",False,False,False,False,Inactive,"['B2B', 'Finance and Accounting']","['United States of America', 'America / Canada...",Early,False,False,False,0,0,Remote
4408,Manatee,"College Park, MD, USA; Rockville, MD, USA; Remote",We let you see everything about how prospects ...,DocSend for demos.,2.0,False,False,False,"['B2B', 'Sales']",False,False,False,False,Inactive,"['B2B', 'Sales']","['United States of America', 'America / Canada...",Early,True,True,False,2,1,"College Park, MD, USA"
4408,Manatee,"College Park, MD, USA; Rockville, MD, USA; Remote",We let you see everything about how prospects ...,DocSend for demos.,2.0,False,False,False,"['B2B', 'Sales']",False,False,False,False,Inactive,"['B2B', 'Sales']","['United States of America', 'America / Canada...",Early,True,True,False,2,1,"Rockville, MD, USA"
4408,Manatee,"College Park, MD, USA; Rockville, MD, USA; Remote",We let you see everything about how prospects ...,DocSend for demos.,2.0,False,False,False,"['B2B', 'Sales']",False,False,False,False,Inactive,"['B2B', 'Sales']","['United States of America', 'America / Canada...",Early,True,True,False,2,1,Remote


In [96]:
df['regions'][0]

"['United States of America', 'America / Canada']"

In [ ]:
df['regions_list']